In [388]:
using JuMP
using GLPK
using DataFrames

In [389]:
demand = DataFrame(period = [1,2,3,4,5,6,7],
    demand = [12000, 32000, 25000, 36000, 25000, 30000, 18000],
    hours = [6,3,3,2,4,4,2]
)

,period,demand,hours
,Int64,Int64,Int64
1,1,12000,6
2,2,32000,3
3,3,25000,3
4,4,36000,2
5,5,25000,4
6,6,30000,4
7,7,18000,2


In [390]:
dat = DataFrame(type = [1,2,3,4],
    available_number = [10, 4, 8, 3],
    min_output = [750, 1000, 1200, 1800],
    max_capacity = [1750, 1500, 2000, 3500],
    fix_cost = [2250, 1800, 3750, 4800],
    additional_cost = [2.7, 2.2, 1.8, 3.8],
    start_up_cost = [5000, 1600, 2400, 1200])

n_types = maximum(dat.type)
n_period = maximum(demand.period)
cope_increase = 0.2


0.2

In [415]:
model = Model(GLPK.Optimizer)

@variable(model, work[1:n_types, 1:n_period] >= 0, Int)
@variable(model, start[1:n_types, 1:n_period] >= 0, Int)
@variable(model, padd[1:n_types, 1:n_period] >= 0, Int)

for i in 1:n_types
    for j in 1:n_period
        @constraint(model, work[i, j] <= dat.available_number[i])
    end
end

for i in 1:n_types
    for j in 2:n_period
        @constraint(model, start[i, j] >= work[i, j] - work[i, j-1])
    end
end

for i in 1:n_types
    @constraint(model, start[i, 1] >= work[i, 1] - work[i, n_period])
end

for j in 1:n_period
    for i in 1:n_types
        @constraint(model, sum(padd[i, j]) <= (dat.max_capacity[i] - dat.min_output[i]) * work[i, j])
    end
end

for j in 1:n_period
    @constraint(model, sum(dat.min_output .* work[:, j] + padd[:, j]) >= demand.demand[j])
end

for j in 1:n_period
    @constraint(model, sum(dat.max_capacity .* work[:, j]) >= 1.2 * demand.demand[j])
end

# added the cost of the initial start-up
# not in the solution --> sum(value.(work[:, 1]) .* dat.start_up_cost)
@objective(model, Min, sum(work[:, 1] .* dat.start_up_cost) + sum(sum(start[i, j] * dat.start_up_cost[i] + demand.hours[j] * (dat.fix_cost[i] * work[i,j] + padd[i, j] * dat.additional_cost[i]) for j in 1:n_period) for i in 1:n_types))

optimize!(model)

In [416]:
JuMP.termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [417]:
JuMP.objective_value(model)

1.48723e6

In [418]:
JuMP.value.(padd)

4×7 Array{Float64,2}:
    0.0  1600.0     0.0  3350.0     0.0  1100.0     0.0
 1450.0  2000.0   200.0  2000.0  1900.0  2000.0  1650.0
 1600.0  6400.0  6400.0  6400.0  5600.0  6400.0  3200.0
    0.0     0.0     0.0     0.0     0.0     0.0     0.0

In [419]:
JuMP.value.(work)

4×7 Array{Float64,2}:
 1.0  4.0  4.0  7.0  2.0  2.0  1.0
 4.0  4.0  4.0  4.0  4.0  4.0  4.0
 2.0  8.0  8.0  8.0  7.0  8.0  4.0
 1.0  3.0  1.0  3.0  2.0  3.0  2.0

In [420]:
JuMP.value.(work) .* dat.min_output .+ JuMP.value.(padd)

4×7 Array{Float64,2}:
  750.0   4600.0   3000.0   8600.0   1500.0   2600.0   750.0
 5450.0   6000.0   4200.0   6000.0   5900.0   6000.0  5650.0
 4000.0  16000.0  16000.0  16000.0  14000.0  16000.0  8000.0
 1800.0   5400.0   1800.0   5400.0   3600.0   5400.0  3600.0

In [421]:
sum(JuMP.value.(work) .* dat.min_output .+ JuMP.value.(padd), dims = 1)

1×7 Array{Float64,2}:
 12000.0  32000.0  25000.0  36000.0  25000.0  30000.0  18000.0

In [422]:
JuMP.value.(start)

4×7 Array{Float64,2}:
 0.0  3.0  0.0  3.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  6.0  0.0  0.0  0.0  1.0  0.0
 0.0  2.0  0.0  2.0  0.0  1.0  0.0